In [1]:
import cv2
from ultralytics import YOLO
import supervision as sv
# from supervision import Detections
import numpy as np
from IPython.display import clear_output

In [2]:
# extract video frame
video_path = "video.mp4"

generator = sv.get_video_frames_generator(video_path)
iterator = iter(generator)
frame = next(iterator)

# save first frame
cv2.imwrite("first_frame.png", frame)

True

In [3]:
# Open the video file
cap = cv2.VideoCapture(video_path)

video_info = sv.VideoInfo.from_video_path(video_path)
frame_rate = video_info.fps
frame_width = video_info.width
frame_height = video_info.height

video_info

VideoInfo(width=1920, height=1080, fps=30, total_frames=302)

In [4]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

In [5]:
# Open the video file
out = cv2.VideoWriter('video-with-counter-result.avi', cv2.VideoWriter_fourcc('M','J','P','G'), frame_rate, (frame_width, frame_height))


colors = sv.ColorPalette.DEFAULT
line_start_point = sv.Point(275,730)
line_end_point = sv.Point(947, 922)
line_zone = sv.LineZone(line_start_point,line_end_point)
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=2, text_scale=1,color=sv.Color.BLUE)


polygon_points = [
np.array([[1407, 6],[31, 862],[27, 18],[1403, 2]]),
np.array([[1807, 6],[787, 1066],[1891, 1062],[1903, 10],[1807, 10]])
]


    
polygon_zones = [sv.PolygonZone(polygon=polygon_point, frame_resolution_wh=video_info.resolution_wh) for polygon_point in polygon_points]
polygon_zone_annotators = [sv.PolygonZoneAnnotator(zone=zone,color=colors.by_idx(index),thickness=4,text_thickness=8,text_scale=2) for index, zone in enumerate(polygon_zones)]
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, classes=[0], persist=True, tracker="bytetrack.yaml")
        frame = results[0].plot()
        detections = sv.Detections.from_ultralytics(results[0])
        

        line_zone.trigger(detections = detections)
        # Visualize the results on the frame
        frame = line_zone_annotator.annotate(frame, line_zone)

        for polygon_zone, polygon_zone_annotator in zip(polygon_zones,polygon_zone_annotators):
            mask = polygon_zone.trigger(detections = detections)
            detections_filtered = detections[mask]
            frame = polygon_zone_annotator.annotate(frame)
        out.write(frame)
        
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
clear_output()